In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Thu Jan 31 15:33:45 2019

@author: Arati
"""
import pandas as pd
from termcolor import colored
##from collections import namedtuple

In [ ]:
concept_set = {'zeroconcept'}
## concept set contains all concept names, including separate entries for synonyms
#%%

def check_exists(conceptname):
    conceptname=refer(conceptname)
    if (conceptname in concept_set):
        return True
    else:
        return False
## maybe add in an additional functiont to check is some similar form is already in the set, and ask to add as synonym.
#%%

In [ ]:
def exists(conceptname):
    conceptname = refer(conceptname)
    if (isinstance(conceptname,str) is False):
        print (colored('concept name is not a string','red'))
        return 0
    elif check_exists(conceptname):
        print (colored(conceptname + ' is already in concept set','cyan'))
        return 1
    else:
        concept_set.add(conceptname)
        print(conceptname + ' has been added to concept set')
        return 1

In [ ]:
#%% plays this role - accepts from a standard set of roles only
role_set = {'set', 'instance','attribute','rule','tool','functor','application','process','step','event','system','component','field','model','symptom','list','agent'}
## set - there exists more than one, it is a subset (unless universe set) that inherits all the attributes of its direct superset with the exception of atleast one on the basis of which its name is different
## instance-  an instance of a set currently under consideration
## attribute - is a property of a set or instance. so if something has an attribute it is either a set or an instance, in addition to other fields.
## rule - it is an assignment to the value of an attribute based on some conditions being true for a particular instance.
## tool - is acknowledged as useful - can be a system, model, rule etc.
## functor - changes the value of atleast one attribute of an instance reversibly or irreversibly
## application - purpose of something, goal. belongs to an agent.
## process - list of steps
## step - instruction with or without conditions attached to it. based on some rule.
## event - has a trigger - which is a condition, and 

## roles stored in dataframe with index number, and two columns, conceptname and role.
roles = pd.DataFrame(columns = ['conceptname','role'])
## or alternately loaded from file
#%%
def check_role(conceptname):
    role_list= []
    conceptname=refer(conceptname)
    if check_exists(conceptname) is False:
        print(colored(conceptname+' does not exist in concept set','red'))
        return role_list
    else:
        role_list = list(roles[roles['conceptname']==conceptname]['role'])
        if (len(role_list) == 0):
            print(colored('No roles added for '+conceptname+' so far.','cyan'))
            return role_list
        else:
            return role_list

#%% 
def plays_role(conceptname1, role1):
    global roles
    conceptname1 = refer(conceptname1)
    if (exists(conceptname1)==1):
        if role1.lower() in role_set:
            if role1 in check_role(conceptname1):
                print(colored(conceptname1+' already plays the role '+role1+', not adding duplicate record','cyan'))
            else:
                roles = roles.append(pd.DataFrame([[conceptname1,role1]],columns=list(roles.columns)),ignore_index=True)
                print(conceptname1+ ' now plays the role '+role1)
        else:
            print(colored('role is not a listed one','red'))    
        

#%%
def delete_role(conceptname1,role1):
    global roles
    conceptname1 = refer(conceptname1)
    if check_exists(conceptname1):
        if role1.lower() in role_set:
            if role1 in check_role(conceptname1):
                index = roles[(roles.conceptname==conceptname1) & (roles.role==role1)].index[0]
                print(conceptname1 + ' currently plays the role '+role1+ '. deleting..')
                roles.drop(index,inplace=True,axis=0)
            else: 
                print(colored('No record found of '+conceptname1+' playing role '+role1,'red'))
        else: 
            print(colored('role is not a listed one','red'))
    else:
        print(colored(conceptname1+ ' does not exist in concept set','red'))
#%%

In [ ]:
        
## attribute list data frame should contain attribute
attributes = pd.DataFrame(columns=['attributename','vartype','mutability','unit_domain_dict','supersetset'])
## vartype takes one of the following values: 

#%%
## if A is a type of B, then all of B's properties and rules are also obeyed for A.
typeof = pd.DataFrame(columns=['subset','superset'])
#%%
def direct_subsets(superset):
    global typeof
    superset= refer(superset)
    subsetmatched = typeof.loc[typeof.superset==superset,'subset']
    return list(subsetmatched)
#%%
def direct_supersets(subset):
    global typeof
    subset = refer(subset)
    supersetmatched = typeof.loc[typeof.subset==subset,'superset']
    return list(supersetmatched)
#%%
def subsets(superset,maxlevel=None):
    allsubsetlist = []
    superset=refer(superset)
    superset = [superset]
    if maxlevel is not None:
        for i in range(maxlevel):
            subsetlist = []
            for item in superset:
                subsetlist = subsetlist + direct_subsets(item)
                ##print(subsetlist)
            if len(subsetlist)>0:
                allsubsetlist = allsubsetlist + subsetlist
                superset=subsetlist
            else:
                return allsubsetlist
    else:
        
        exit_flag=0
        while exit_flag==0:
            subsetlist = []
            for item in superset:
                subsetlist = subsetlist + direct_subsets(item)
                ##print(subsetlist)
            if len(subsetlist)>0:
                allsubsetlist = allsubsetlist + subsetlist
                superset=subsetlist
            else:
                return allsubsetlist
                exit_flag = 1
    return allsubsetlist
#%%
def supersets(subset,maxlevel=None):
    allsupersetlist = []
    subset = refer(subset)
    subset = [subset]
    if maxlevel is not None:
        for i in range(maxlevel):
            supersetlist = []
            for item in subset:
                supersetlist = supersetlist + direct_supersets(item)
                ##print(subsetlist)
            if len(supersetlist)>0:
                allsupersetlist = allsupersetlist + supersetlist
                subset=supersetlist
            else:
                return allsupersetlist
    else:
        
        exit_flag=0
        while exit_flag==0:
            supersetlist = []
            for item in subset:
                supersetlist = supersetlist + direct_supersets(item)
                ##print(subsetlist)
            if len(supersetlist)>0:
                allsupersetlist = allsupersetlist + supersetlist
                subset=supersetlist
            else:
                return allsupersetlist
                exit_flag = 1
    return allsupersetlist
#%%     
#%%
def is_type_of(subset,superset):
    global typeof
    subset= refer(subset)
    superset = refer(superset)
    ##exists(subset)
    ##exists(superset)
    ## check if subset is a subset of any mutually exclusive set with superset - if so, then you cannot make this a subset
    if superset in subsets(subset):
        print(colored(superset+' is already a subset of '+subset+', please resolve.','red'))
    elif subset in supersets(superset):
        print(colored(subset+' is already a superset of '+superset+', please resolvee.','red'))
    elif typeof[(typeof.superset==superset) & (typeof.subset==subset)].empty is False:
        print(colored('record already added, not adding duplicate record','cyan'))
    else:
        plays_role(subset,'set')
        plays_role(superset,'set')
    
        typeof = typeof.append(pd.DataFrame([[subset,superset]],columns=['subset','superset']),ignore_index=True,sort=True)
        print('added record')
    
    ## all attributes of superset and attributes of subset
    ## all rules obeyed by superset are obeyed by subset?
    ## all roles played by superset are obeyed by subset
    ## all attributes of subset are automatically attributes of superset (sans value range)
    ## all rules, roles played by subset, are obeyed by superset
    
#%%
#%%
def set_attribute_details(attributename,vartype,mutability=None,unit_domain_dict=None,supersetset=None):
    attributename = refer(attributename)
    
    if vartype=='categorical':
        if len(get_depvariables(attributename))>0:
            print(colored('Cannot set vartype to categorical variable if proportionality rule exists, please resolve','red'))
            print(colored('The following variables are dependent on '+attributename,'red'))
            print(colored(get_depvariables(attributename),'red'))
            return 0
        if len(get_indvariables(attributename))>0:
            print(colored('Cannot set vartype to categorical variable if proportionality rule exists, please resolve','red'))
            print(colored('The following variables are dependent on '+attributename,'red'))
            print(colored(get_indvariables(attributename),'red'))
            return 0
            
   
    if isinstance(unit_domain_dict,dict):
        unit_domain_dict = [unit_domain_dict]
    global attributes
    args = locals()
    print(args)
    attvartypematched = attributes[ (attributes.attributename==attributename) & (attributes.vartype==vartype)]
    attvartypenotmatched = attributes[(attributes.attributename==attributename) & (attributes.vartype!=vartype)]
    no_matchedrec = attvartypematched.empty
    no_unmatchedrec = attvartypenotmatched.empty
    
    if no_unmatchedrec is False:
        print(colored('attribute is already present as another vartype:'+str(attvartypenotmatched.vartype)+'. Please resolve.','red'))
        return 0
    elif no_matchedrec is False:
        print('found a record for this attribute already, editing record...')
        for k,v in args.items():
            if v is not None:
                print(k+': '+str(v))
                attributes.loc[(attributes.attributename==attributename) & (attributes.vartype==vartype),k] = v
        return 1
    else:
        print('no record found, adding')
        attributes = attributes.append(pd.DataFrame([[v for k,v in args.items() if v is not None]],columns=[k for k,v in args.items() if v is not None]),ignore_index=True,sort=True)
        return 1
#%%
def get_attribute_details(attributename,fieldlist):
    attributename = refer(attributename)
    global attributes
    if type(fieldlist)!=list:
        fieldlist = [fieldlist]
    for field in fieldlist:
        if field not in fieldlist:
            print(colored(field + ' is not a valid field','cyan'))
            return []
    listofatts= attributes.loc[attributes.attributename==attributename,fieldlist]
    if listofatts.empty:
        return pd.DataFrame()
    else:
        return listofatts
#%%
## states is a table of attribute values for all concepts 
## fields are conceptname, attribute, value, units, probability distribution, (for now) 
states = pd.DataFrame(columns=['conceptname','attributename','value','units', 'confidence'])
## need to decide on either an expectation value of each value or a discrete probability number
#%%

In [ ]:
import numpy as num
def normalize(listpercentage):
    listpercentage = num.array(listpercentage)
    listpercentagen = listpercentage/sum(listpercentage)
    return list(listpercentagen)
#%%
def is_attribute_of(attributename,conceptname,value=None,units=None,confidence=None):
    global states
    attributename = refer(attributename)
    conceptname = refer(conceptname)
    args = locals()
    columnslist = [k for k,v in args.items() if v is not None]
    print(columnslist)
    argslist = [v for k,v in args.items() if v is not None]
    print(argslist)
    3#exists(attributename)
    exists(conceptname)
    plays_role(attributename,'attribute')
    
    ## okay, the decision is, AN ATTRIBUTE can only have one vartype.
    ## so check if attribute details are set, then validate.
    ## otherwise set attribute details and add to records
    matchedstates = states[(states.conceptname ==conceptname) & (states.attributename==attributename)]
    if matchedstates.empty:
        states = states.append(pd.DataFrame([argslist],columns=columnslist),ignore_index=True,sort=True)
    else:
        if value is not None and units is not None:
            states[(states.conceptname ==conceptname) & (states.attributename==attributename)].loc[states.value]=value
            states[(states.conceptname ==conceptname) & (states.attributename==attributename)].loc[states.units]=units
            if confidence is None:
                states[(states.conceptname ==conceptname) & (states.attributename==attributename)].loc[states.confidence]=100
            else:
                states[(states.conceptname ==conceptname) & (states.attributename==attributename)].loc[states.confidence]=confidence
            states[(states.conceptname ==conceptname) & (states.attributename==attributename)].loc[states.confidence] = normalize(matchedstates.loc[states.confidence])

#%%

In [ ]:
components = pd.DataFrame(columns=['compname','sysname'])
#%%
def direct_components(systemname):
    ## check if systemname is a component of compname
    ## if not, you can add.
    systemname = refer(systemname)
    global components
    matched = components.loc[components.sysname==systemname,'compname']
    return list(matched)
#%%
def direct_systems(compname):
    compname = refer(compname)
    global components
    matched = components.loc[components.compname==compname,'sysname']
    return list(matched)
#%%
def all_components(systemname,maxlevel=None):
    allcomponentlist = []
    systemname = refer(systemname)
    systemname = [systemname]
    if maxlevel is not None:
        for i in range(maxlevel):
            componentlist = []
            for item in systemname:
                componentlist = componentlist + direct_components(item)
                ##print(subsetlist)
            if len(componentlist)>0:
                allcomponentlist = allcomponentlist + componentlist
                systemname=componentlist
            else:
                return allcomponentlist
    else:
        
        exit_flag=0
        while exit_flag==0:
            componentlist = []
            for item in systemname:
                componentlist = componentlist + direct_components(item)
                ##print(subsetlist)
            if len(componentlist)>0:
                allcomponentlist = allcomponentlist + componentlist
                systemname=componentlist
            else:
                return allcomponentlist
                exit_flag = 1
    return allcomponentlist
#%%
def is_component_of(component,system):
    global components
    component=refer(component)
    system = refer(system)
    ##exists(subset)
    ##exists(superset)
    if system in all_components(component):
        print(colored(system+' is already a component of '+component+', not adding. Please resolve.','red'))
    elif component in all_systems(system):
        print(component+' is a system containing component '+system+', not adding.')
    elif components[(components.sysname==system) & (components.compname==component)].empty is False:
        print(colored('record already added, not adding duplicate record','cyan'))
    else:
        plays_role(component,'component')
        plays_role(system,'system')
        components = components.append(pd.DataFrame([[component,system]],columns=['compname','sysname']),ignore_index=True,sort=True)
        print('added record')
    
    ## all attributes of superset and attributes of subset
    ## all rules obeyed by superset are obeyed by subset?
    ## all roles played by superset are obeyed by subset
    ## all attributes of subset are automatically attributes of superset (sans value range)
    ## all rules, roles played by subset, are obeyed by superset

#%%
def all_systems(component,maxlevel=None):
    allsystemlist = []
    component = refer(component)
    component = [component]
    if maxlevel is not None:
        for i in range(maxlevel):
            systemlist = []
            for item in component:
                systemlist = systemlist + direct_systems(item)
                ##print(subsetlist)
            if len(systemlist)>0:
                allsystemlist = allsystemlist + systemlist
                component=systemlist
            else:
                return allsystemlist
    else:
        
        exit_flag=0
        while exit_flag==0:
            systemlist = []
            for item in component:
                systemlist = systemlist + direct_systems(item)
                ##print(subsetlist)
            if len(systemlist)>0:
                allsystemlist = allsystemlist + systemlist
                component=systemlist
            else:
                return allsystemlist
                exit_flag = 1
    return allsystemlist
#%%

def list_attributes_of(conceptname):
    conceptname = refer(conceptname)
    listofatts = states.loc[states.conceptname==conceptname,:]
    return listofatts
        
#%%
def findallinformation(conceptname):
    conceptname = refer(conceptname)
    allinfo = {}
    if check_exists(conceptname):
        role_list = check_role(conceptname)
        
        if len(role_list)>0:
            print(conceptname + ' plays the following roles: ')
            print(role_list)
            allinfo['role_list']=role_list
        
        if 'set' in role_list:
            subsetlist = subsets(conceptname)
            if len(subsetlist)>0:
                print(conceptname + ' has the following subsets:')
                print(subsetlist)
                allinfo['subsetlist'] = subsetlist
            supersetlist = supersets(conceptname)
            if len(supersetlist)>0:
                print(conceptname+ ' has the following supersets')
                print(supersetlist)
                allinfo['supersetlist']=supersetlist
            
        if 'attribute' in role_list:
            attdeets = attributes[(attributes.attributename==conceptname)]
            if attdeets.empty is False:
                allinfo['attdeets'] = attdeets
                print(conceptname+ ' is an attribute with the following characteristics')
                print(attdeets)
            
            indvar = get_indvariables(conceptname)
            depvar = get_depvariables(conceptname)
            if len(indvar)>0:     
                if len(indvar[0])>0:
                    print(conceptname+' depends on the following variables')
                    vars = [k for k,v in indvar[0].items()]
                    eff = [v for k,v in indvar[0].items()]
                    for i in range(len(indvar[0])):
                        if v==1:
                            print('As '+k+' increases,'+conceptname+' increases in system: '+indvar[1][i])
                        else:
                            print('As '+k+' increases,'+conceptname+' decreases n system: '+indvar[1][i])
            
            if len(depvar)>0:
                if len(depvar[0])>0:
                    print(conceptname+' impacts the following variables')
                    vars = [k for k,v in depvar[0].items()]
                    eff = [v for k,v in depvar[0].items()]
                    
                    for i in range(len(depvar[0])):
                        if v==1:
                            print('As '+k+' increases,'+conceptname+' increases in system: '+depvar[1][i])
                        else:
                            print('As '+k+' increases,'+conceptname+' decreases n system: '+depvar[1][i])
        
        if 'functor' in role_list:
            functordeets = functors[(functors.functorname==conceptname)]
            if functordeets.empty is False:
                allinfo['functordeets']=functordeets
                print(conceptname+' is a functor with teh following characteristics')
                print(functordeets)
        
        if 'application' in role_list:
            appdeets = applications[(applications.application==conceptname)]
            if appdeets.empty is False:
                allinfo['appdeets']= appdeets
                print(conceptname + ' is an application for which the following tools may be useful')
                print(appdeets)
        
        if 'tool' in role_list:
            tooldeets = applications[(applications.toolname==conceptname)]
            if tooldeets.empty is False:
                allinfo['tooldeets'] = tooldeets
                print(conceptname+' is a tool with the following listed applications.' )
                print(tooldeets)
        
        ##listoffuncts = functors[]
        
        listofatts = list_attributes_of(conceptname)
        if listofatts.empty is False:
            allinfo['listofatts'] = listofatts
            print('The state of '+conceptname+' is currently as follows:')
            print(listofatts)
            
        if 'system' in role_list:
            componentlist = all_components(conceptname)
            if len(componentlist)>0:
                print(conceptname + ' has the following components')
                print(componentlist)
                allinfo['componentlist'] = componentlist
        
            
        if 'component' in role_list:
            systemlist = all_systems(conceptname)
            if len(systemlist)>0:
                print(conceptname + ' is a component for the following systems')
                print(systemlist)
                allinfo['systemlist'] = systemlist
        
        return allinfo
    else:
        print(conceptname + ' does not exist')
        return {}        
    
#%%
synonym_dict = {}
#%%
def is_aka(setnames):
    setnames = set(setnames)
    global synonym_dict
    ## check which of the names has attributes/states/ etc. associated with it.
    ## print those out and ask to resolve.
    ## none of these names exist, then add name0 as key andd others as synonyms
    ## if non name0 name already has rules associated with it, and no other names including name0
    ## maybe key is synonyms and value is real name.
    ## in exists function, add to synonym table. 
    
    
    
    count = []
    infodict = {}
    for item in setnames:
        infodict[item] = findallinformation(item)
        if len(infodict[item])>0:
            count.append(item)
    
    if len(count)>1:
        print('The following names have descriptions that already exist, please resolve')
        print([key for (key,value) in infodict.items() if len(value)>0])
    elif len(count)==1:
        print('The name '+count[0]+' already has facts and rules associated with it, it will be name of record and others will act as synonyms' )
        for item in setnames:
            if item not in count:
                synonym_dict[item]=count[0]
    elif len(count)==0:
        print('none of these names have descriptors yet, adding name '+list(setnames)[0]+' as name of record, rest will be synonyms')
        exists(list(setnames)[0])
        for item in setnames:
            if item!=list(setnames)[0]:
                synonym_dict[item]=list(setnames)[0]

        
#%%
def refer(name):
    global synonym_dict
    name1 = synonym_dict.get(name,name)
    if name1!=name:
        print(name+ ' is a synonym for '+name1)
    return name1
#%% 
functors = pd.DataFrame(columns=['functorname','inputslist','outputslist','destructive','actualfunctionname'])

#%%
def is_a_functor(functorname,supersystem,outputset=None,inputset=None,destructive=None,actualfunctionname=None):
    ## inputs are all attributes of something
    ## outputs are attributes of same thing or different thing in same system
    ## destructive tells us if inputslist is modified by action of functor or remains as is
    ## actualfunctionname is for a written python function that does the functor
    global functors
    args=locals()
    print(args)
    functorname = refer(functorname)
    plays_role(functorname,'functor')
    supersystem = refer(supersystem)
    plays_role(supersystem,'system')
    isattin = []
    isattout = []
    for inp in inputset:
        inp1 = refer(inp)
        if(inp1!=inp):
            inputset.add(inp1)
            inputset.remove(inp)
        plays_role(inp1,'attribute')
        is_attribute_of(inp1,supersystem)
    for oup in outputset:
        oup1 = refer(oup)
        if(oup1!=oup):
            outputset.add(oup1)
            outputset.remove(oup)
        plays_role(oup,'attribute')
        is_attribute_of(oup1,supersystem)
    
    
    ## functor by this name already exists, making changes
    
    functors[functors.functorname==functorname]
    if functors[functors.functorname==functorname].empty is False:
        print('functor by name '+functorname+' already exists. Editing...')
        if outputset is not None:
            functors.loc[functors.functorname==functorname,functors.outputset].union(outputset)
        if inputset is not None:
            functors.loc[functors.functorname==functorname,functors.inputset].union(inputset)
        if destructive is not None:
            functors.loc[functors.functorname==functorname,functors.destructive] = destructive
        if actualfunctionname is not None:
            functors.loc[functors.functorname==functorname,functors.actualfunctionname] = actualfunctionname
    
    ## functor by this name does not exist
    valslist = [v for k,v in args.items() if v is not None]
    print(valslist)
    colslist = [k for k,v in args.items() if v is not None]
    print(colslist)
    print('Adding record')
    functors = functors.append(pd.DataFrame([valslist],columns=colslist),ignore_index=True,sort=False)
    
#%%    
## is useful for
applications = pd.DataFrame(columns=['toolname' , 'application'])
#%%
def is_useful_for(toolname,application):
    global applications
    toolname = refer(toolname)
    application= refer(application)
    ## check for duplicate record, otherwise add
    plays_role(application,'application')
    plays_role(toolname,'tool')
    appsub = applications[(applications.toolname == toolname) & (applications.application == application)]
    if appsub.empty:
        print('Adding record.')
        applications = applications.append(pd.DataFrame([[toolname,application]],columns=['toolname','application']),ignore_index=True,sort=False)
    else: 
        print('Record already exists, not adding duplicate record')
#%%
prop=pd.DataFrame(columns=['dep_attribute','ind_attribute','supersystem','effect'])
#%%
def proportionality_rule(dep_attribute,ind_attribute,supersystem,effect=None):
    ## effect +1: dep inc as ind inc
    ## effect -1: dep dec as ind inc
    args = locals()
    global prop
    vals = [v for k,v in args.items() if v is not None]
    keys = [k for k,v in args.items() if v is not None]
    ind_attribute = refer(ind_attribute)
    dep_attribute = refer(dep_attribute)
    
    supersystem = refer(supersystem)
    plays_role(supersystem,'system')
    
    if (dep_attribute==ind_attribute):
        print('Dependent attribute same as independent attribute, not adding')
    if (dep_attribute==supersystem):
        print(supersystem + ' cannot be an attribute of itself, not adding')
    if (ind_attribute==supersystem):
        print(supersystem + ' cannot be an attribute of itself, not adding')
    
    is_attribute_of(ind_attribute,supersystem)
    is_attribute_of(dep_attribute,supersystem)
    
    ## the attributes have to be continuous, discrete or ordinal. they cannot be categorical. 
    inddet = get_attribute_details(ind_attribute,'vartype')
    depdet = get_attribute_details(dep_attribute,'vartype')
    if inddet.empty is False:
        if inddet.loc[:,'vartype'][0]=='categorical':
            print('Independent variable is of categorical type. Proportionality rules apply only to continuous, discrete numeric or ordinal variables')

    if depdet.empty is False:
        if depdet.loc[:,'vartype'][0]=='categorical':
            print('Dependent variable is of categorical type. Proportionality rules apply only to continuous, discrete numeric or ordinal variables')
        
    
    ##checking for duplicate record
    propsub = prop[(prop.ind_attribute==ind_attribute) & (prop.dep_attribute==dep_attribute)]
    if propsub.empty is False:
        
        if effect is not None:
            prop.loc[(prop.ind_attribute==ind_attribute) & (prop.dep_attribute==dep_attribute) & (prop.supersyste==supersystem),prop.effect] = effect
            if effect==1:
                print('As '+ind_attribute+' increases, '+dep_attribute+' also increases: in '+supersystem)
            else:
                print('As '+ind_attribute+' increases, '+dep_attribute+' decreases: in '+supersystem)
        else:
            print(dep_attribute+' depends on '+ind_attribute+' :in '+supersystem)
    else:
        if effect is not None:
            prop = prop.append(pd.DataFrame([vals],columns=keys),ignore_index=True,sort=False)
            if effect==1:
                print('As '+ind_attribute+' increases, '+dep_attribute+' also increases: in '+supersystem)
            else:
                print('As '+ind_attribute+' increases, '+dep_attribute+' decreases: in '+supersystem)
        else:
            print(dep_attribute+' depends on '+ind_attribute+' :in '+supersystem)
            

#%%
def get_depvariables(attributename):
    
    global prop
    attributename = refer(attributename)
    propsub = prop[prop.ind_attribute==attributename]
    
    if propsub.empty is False:
        depvar = list(propsub.dep_attribute)
        efflist = list(propsub.effect)
        return [dict(zip(depvar,efflist)),list(propsub.supersystem)]
    else:
        return []
    
#%%
def get_indvariables(attributename):
    
    global prop
    attributename = refer(attributename)
    propsub = prop[prop.dep_attribute==attributename]
    print(propsub)
    if propsub.empty is False:
        indvar = list(propsub.ind_attribute)
        efflist = list(propsub.effect)
        return [dict(zip(indvar,efflist)), list(propsub.supersystem)]
    else:
        return []
#%%
def mutually_exclusive_and_all_inclusive_sets(setlist,superset, basisattribute):
    for item in setlist:
        item1 = refer(setlist)
        